In [15]:
# Topological Art Generator using Seifert Surfaces & Vector Fields
# Author: Carys Garvey (Summer 2025 Topology Research)

# --- Section 1: Imports and Setup ---
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import Normalize
from scipy.spatial import KDTree
import ipywidgets as widgets
from IPython.display import display

# These imports allow:
# - np: numerical operations
# - plt: visualization of surfaces and vector fields
# - cm, Normalize: color mapping for scalar fields like divergence
# - KDTree: for nearest-neighbor queries (e.g., for extensions or projections)
# - widgets: interactive parameter controls in Jupyter
# - display: to render widgets cleanly


In [16]:
# ------------------------------------------------------------------------------
# torus_knot(p, q, theta, r)
#
# Generates a smooth surface bounded by the (p, q) torus knot.
#
# This function constructs a Seifert-like surface by linearly interpolating
# between the core circle of a torus and the torus knot that wraps around it.
#
# Geometric Intuition:
# - The "centerline" is the circular path through the middle of the torus — the curve
#   traced by the center of the torus tube. In a top-down view, it looks like the
#   large circular loop of the donut hole.
# - The (p, q) torus knot wraps p times longitudinally (around the hole) and
#   q times meridionally (around the tube).
# - The surface forms a twisted, disk-like annulus connecting this centerline
#   to the knot curve, sweeping out a 2D manifold with the knot as its boundary.
#
# Topological Note:
# - The output is an embedded, orientable surface in ℝ³ whose boundary is the (p, q) knot.
# - It is thus a valid Seifert surface — though not necessarily of minimal genus.
#   It satisfies the key topological properties needed for theoretical and visual exploration.
#
# Parameters:
# - p, q: integers defining the torus knot type
# - theta: angular parameter along the knot (from 0 to 2π)
# - r: radial parameter (0 = centerline, 1 = knot boundary)
#
# Returns:
# - x, y, z: coordinates of surface points in ℝ³
# ------------------------------------------------------------------------------

def torus_knot(p, q, theta, r):
    
    R, r0 = 2.0, 0.7  # R: major radius of torus; r0: tube radius
    phi = p * theta   # angular winding around torus core
    psi = q * theta   # angular twisting along tube

    # Boundary curve of the knot
    x0 = (R + r0 * np.cos(psi)) * np.cos(phi)
    y0 = (R + r0 * np.cos(psi)) * np.sin(phi)
    z0 = r0 * np.sin(psi)

    # Center ring of the torus
    x_center = R * np.cos(phi)
    y_center = R * np.sin(phi)
    z_center = np.zeros_like(phi)

    # Linear interpolation from torus core (r=0) to knot (r=1)
    x = r * x0 + (1 - r) * x_center
    y = r * y0 + (1 - r) * y_center
    z = r * z0 + (1 - r) * z_center

    return x, y, z


In [17]:
# ------------------------------------------------------------------------------
# get_surface_parametrization(surface_id, theta, r)
#
# Dispatcher function that maps a surface ID string (e.g., 'T(2,3)', '3_1', '8_1')
# to a specific parametric surface formula that generates Seifert-like surfaces
# bounded by knots or artistic loops.
#
# Each surface is defined via:
#   - theta: angular coordinate (0 to 2π) tracing the boundary knot
#   - r: radial coordinate (0 to 1) interpolating from a central curve (like a torus core)
#        out to the knot boundary, forming a smooth annular surface.
#
# Many of these surfaces visually and topologically behave like Seifert surfaces:
# - They are connected, embedded 2D manifolds in ℝ³
# - Their boundary traces out the specified knot
#
# Surface Types:
# - Torus knots like T(p,q) use standard parametric equations (e.g., from MathWorld)
# - Named knots (e.g., '3_1', '5_1', etc.) use approximated embeddings adapted
#   from known visualizations such as:
#     • Wolfram MathWorld: https://mathworld.wolfram.com/TorusKnot.html
#     • KnotPlot software (Robert Scharein)
#     • Lamm, C. (2000), "Fourier knots and computational geometry"
#     • Hoste et al., "The First 1,701,936 Knots", and the KnotInfo database
#
# These forms are selected for aesthetic clarity and smoothness rather than minimal-genus guarantees.
# This function is useful for both mathematical exploration and generative topological art.
# ------------------------------------------------------------------------------

def get_surface_parametrization(surface_id, theta, r):
    # Torus knot family (classic examples of nontrivial knots)
    if surface_id == 'T(2,3)': return torus_knot(2, 3, theta, r)  # Trefoil
    elif surface_id == 'T(3,5)': return torus_knot(3, 5, theta, r)
    elif surface_id == 'T(4,7)': return torus_knot(4, 7, theta, r)
    elif surface_id == 'T(5,8)': return torus_knot(5, 8, theta, r)
    elif surface_id == 'T(6,11)': return torus_knot(6, 11, theta, r)
    elif surface_id == 'T(7,10)': return torus_knot(7, 10, theta, r)

    # Manually crafted parametric knots with r-scaling
    elif surface_id == '3_1':  # Trefoil with custom formula
        x = r * (np.sin(theta) + 2 * np.sin(2 * theta))
        y = r * (np.cos(theta) - 2 * np.cos(2 * theta))
        z = r * (-np.sin(3 * theta))
    elif surface_id == '4_1':  # Figure-eight knot
        x = r * (2 + np.cos(2 * theta)) * np.cos(3 * theta)
        y = r * (2 + np.cos(2 * theta)) * np.sin(3 * theta)
        z = r * np.sin(4 * theta)
    elif surface_id == '5_1':  # Cinquefoil knot (5-crossings)
        x = r * (np.sin(theta) + 1.8 * np.sin(5 * theta))
        y = r * (np.cos(theta) - 1.8 * np.cos(5 * theta))
        z = r * np.sin(2 * theta)
    elif surface_id == '5_2':  # 3-twist knot variant
        x = r * (10 * np.cos(theta) + np.cos(3 * theta))
        y = r * (6 * np.sin(theta) - np.sin(3 * theta))
        z = r * np.sin(4 * theta)
    elif surface_id == '6_1':  # Stevedore-style knot (6-crossings)
        x = r * (np.sin(theta) + 1.5 * np.sin(3 * theta))
        y = r * (np.cos(theta) - 1.5 * np.cos(3 * theta))
        z = r * np.cos(3 * theta)
    elif surface_id == '7_1':  # High-symmetry 7-petal knot
        x = r * (np.sin(theta) + 1.2 * np.sin(7 * theta))
        y = r * (np.cos(theta) - 1.2 * np.cos(7 * theta))
        z = r * np.sin(3 * theta)
    elif surface_id == '8_1':  # Experimental 8-crossings
        x = r * (np.sin(theta) + 2 * np.sin(4 * theta))
        y = r * (np.cos(theta) - 2 * np.cos(4 * theta))
        z = r * np.sin(5 * theta)
    elif surface_id == '10_1':  # Lobed symmetric construction
        x = r * np.sin(theta) * (1 + 0.5 * np.cos(10 * theta))
        y = r * np.cos(theta) * (1 + 0.5 * np.sin(10 * theta))
        z = r * np.sin(3 * theta)
    elif surface_id == '12_1':  # Star-shaped variant
        x = r * np.cos(theta) * (1 + 0.7 * np.cos(5 * theta))
        y = r * np.sin(theta) * (1 + 0.7 * np.cos(5 * theta + np.pi / 4))
        z = r * np.sin(5 * theta)
    else:
        raise ValueError(f"Unsupported surface ID: {surface_id}")

    return x, y, z


In [18]:
# ------------------------------------------------------------------------------
# custom_surface(theta, r)
#
# Example of a user-defined parametric surface.
#
# This example surface defines a spiraling, twisted paraboloid using polar coordinates.
# It’s not bounded by a knot, but serves as an artistic and mathematical example
# of a smooth 2D manifold embedded in ℝ³.
#
# Geometry:
# - The (x, y) coordinates form a radial disk using polar coordinates.
# - The z-height grows quadratically in r (paraboloid-like) and twists
#   with a cosine-modulated frequency in theta, producing helical ridges.
#
# Topological Context:
# - This is a smooth, connected 2D surface without boundary (if extended fully).
# - It is not a Seifert surface, since it does not bound a knot, but it still qualifies
#   as a parametrized 2-manifold — useful for visualization or testing vector fields on curved domains.
#
# Parameters:
# - theta: angular coordinate (0 to 2π)
# - r: radial coordinate (0 to 1)
#
# Returns:
# - x, y, z: coordinates of the surface in ℝ³
# ------------------------------------------------------------------------------
def custom_surface(theta, r):
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    z = r**2 * np.cos(3 * theta)
    return x, y, z


In [19]:
# ------------------------------------------------------------------------------
# compute_surface_normals_and_basis(param_func, theta, r)
#
# Computes geometric structure on a parametrized surface:
#   - Tangent vectors ∂X/∂θ and ∂X/∂r (Jacobian basis)
#   - Surface normals via the cross product of tangent vectors
#
# This gives the local differential structure of a 2-manifold embedded in ℝ³,
# which is required for defining vector fields on the surface and projecting
# fields into its tangent bundle.
#
# Mathematical Context:
# - Tangent vectors span the tangent space Tₚ(S) at each point p on the surface S.
# - The normal vector is perpendicular to the tangent plane and used to construct
#   orthogonal projections.
# - This procedure defines a local chart, and the surface acts as a smooth 2-manifold.
#
# References:
# - Do Carmo, M. P., "Differential Geometry of Curves and Surfaces", Sections 2–3
# - Lee, J. M., "Introduction to Smooth Manifolds", Ch. 3 (Charts and Parametrizations)
# - Hirsch, M., "Differential Topology", for chart-based modeling of smooth manifolds
#
# Inputs:
# - param_func: function of (theta, r) → (x, y, z)
# - theta, r: meshgrid arrays of angular and radial coordinates
#
# Outputs:
# - normals: unit normal vector at each surface point
# - dx_dtheta, dx_dr: Jacobian basis vectors (∂X/∂θ and ∂X/∂r)
# - x, y: 2D coordinate arrays for plotting
# ------------------------------------------------------------------------------
def compute_surface_normals_and_basis(param_func, theta, r):
    eps = 1e-5  # finite difference step
    x, y, z = param_func(theta, r)
    x_theta, y_theta, z_theta = param_func(theta + eps, r)
    x_r, y_r, z_r = param_func(theta, r + eps)

    dx_dtheta = np.stack([(x_theta - x) / eps,
                          (y_theta - y) / eps,
                          (z_theta - z) / eps], axis=-1)
    dx_dr = np.stack([(x_r - x) / eps,
                      (y_r - y) / eps,
                      (z_r - z) / eps], axis=-1)

    normals = np.cross(dx_dtheta, dx_dr)
    norms = np.linalg.norm(normals, axis=-1, keepdims=True)
    normals /= (norms + 1e-8)
    return normals, dx_dtheta, dx_dr, x, y


In [20]:
# ------------------------------------------------------------------------------
# VECTOR FIELD GENERATORS ON SURFACES
#
# These functions define vector fields in the intrinsic coordinate system
# (θ, r) of a parametrized 2-manifold. The components (v_θ, v_r) describe
# the field in the surface's parameter domain.
#
# After construction, the field is pushed forward to ℝ³ using the Jacobian
# basis (computed from ∂X/∂θ and ∂X/∂r).
#
# Mathematical Context:
# - These fields are sections of the tangent bundle T(S) over a surface S ⊂ ℝ³.
# - The pushforward defines a field of tangent vectors embedded in ℝ³.
# - Used in visualizing flows, divergence, curl, and helicity on surfaces.
#
# References:
# - Lee, J. M., "Introduction to Smooth Manifolds", Ch. 3, 8 (Vector fields, tangent bundles)
# - Abraham, Marsden, Ratiu, "Manifolds, Tensor Analysis, and Applications"
# - Arnold & Khesin, "Topological Methods in Hydrodynamics"
# - Cantarella, DeTurck, Gluck, "Vector Calculus and the Geometry of Curl and Divergence"
# ------------------------------------------------------------------------------

def radial_vector_field(x, y, z):
    """
    Constructs normalized radial vectors from the origin to each point (x, y, z).
    These vectors are later projected onto the tangent plane of the surface.
    """
    vectors = np.stack([x, y, z], axis=-1)
    norms = np.linalg.norm(vectors, axis=-1, keepdims=True)
    vectors /= (norms + 1e-8)
    return vectors

def project_to_surface(vectors, normals):
    """
    Projects 3D vectors onto the surface tangent plane by removing the
    normal component. Equivalent to orthogonal projection in ℝ³.
    """
    dot_products = np.sum(vectors * normals, axis=-1, keepdims=True)
    projected = vectors - dot_products * normals
    return projected

# --- Parametric vector fields in (θ, r) coordinates ---

def field_petal(theta, r, n=6):
    """
    Generates an n-fold symmetric 'petal' field using trigonometric oscillation.
    Illustrates periodic angular structure.
    """
    v_theta = np.sin(n * theta) * r
    v_r = np.cos(n * theta) * r
    return v_theta, v_r

def field_shear(theta, r):
    """
    Constant tangential flow that increases with radial distance.
    Models shear-like angular motion.
    """
    v_theta = np.ones_like(r)
    v_r = np.zeros_like(r)
    return v_theta, v_r

def field_gaussian_vortex(theta, r):
    """
    Localized swirling field centered at r = 0.
    The Gaussian envelope causes decay of magnitude with radius.
    """
    envelope = np.exp(-10 * r**2)
    v_theta = envelope * np.sin(theta)
    v_r = envelope * np.cos(theta)
    return v_theta, v_r

def field_twist(theta, r):
    """
    Twist field oscillating between radial and tangential components.
    Simulates topological twisting effects.
    """
    v_theta = np.sin(4 * theta) * r
    v_r = np.cos(2 * theta) * r
    return v_theta, v_r

def field_radial(theta, r):
    """
    Pure radial vector field: all flow moves outward from center.
    Returned in (v_theta, v_r) format.
    """
    return np.zeros_like(r), np.ones_like(r)

def field_vortex(theta, r):
    """
    Pure angular field: circulation around the origin.
    Tangential only, with magnitude growing linearly in r.
    """
    v_theta = 2 * r
    v_r = np.zeros_like(r)
    return v_theta, v_r

def field_spiral(theta, r):
    """
    Spiral vector field with equal radial and angular components.
    Models outward rotating flow.
    """
    v_theta = r
    v_r = r
    return v_theta, v_r

# ------------------------------------------------------------------------------
# pushforward_parametric_field(v_theta, v_r, dx_dtheta, dx_dr)
#
# Pushes a vector field defined in (θ, r) coordinates forward to ℝ³ using
# the tangent vectors ∂X/∂θ and ∂X/∂r of the surface.
#
# This maps a 2D vector in the parameter domain to a 3D vector on the
# embedded manifold by using the surface's differential (Jacobian).
#
# References:
# - Lee, J. M., "Introduction to Smooth Manifolds", Ch. 3, 8 (Pushforwards, vector fields)
# - Pressley, A., "Elementary Differential Geometry"
# ------------------------------------------------------------------------------
def pushforward_parametric_field(v_theta, v_r, dx_dtheta, dx_dr):
    return v_theta[..., None] * dx_dtheta + v_r[..., None] * dx_dr


In [21]:
# ------------------------------------------------------------------------------
# compute_projected_divergence(param_func, theta_grid, r_grid, field_type)
#
# Projects a vector field defined on a surface into ℝ³ and computes an
# approximate divergence scalar field over the surface.
#
# Steps:
# 1. Compute local tangent basis vectors and surface normals.
# 2. Define or retrieve the specified vector field in either:
#    - (x, y, z) coordinates for radial fields, or
#    - (θ, r) coordinates for parametric fields.
# 3. Push the vector field forward to 3D and decompose it along the local
#    Jacobian basis (dX/dθ, dX/dr).
# 4. Use discrete gradient approximations to compute surface divergence:
#    div(V) ≈ ∂V_θ/∂θ + ∂V_r/∂r
#
# Mathematical Context:
# - This divergence operator is the intrinsic divergence on the surface,
#   projected through the parametrization.
# - The approximation assumes uniform grid spacing in θ and r.
# - Resulting divergence field is a scalar function that captures local
#   expansion (positive) or contraction (negative) of the flow.
#
# References:
# - Lee, J. M., "Introduction to Smooth Manifolds", Ch. 8 (Differential operators)
# - Cantarella et al., "The Geometry of the Curl Operator"
# - Abraham, Marsden, Ratiu, "Manifolds, Tensor Analysis, and Applications"
# - Moffatt, H. K., "The degree of knottedness of tangled vortex lines"
# ------------------------------------------------------------------------------

def compute_projected_divergence(param_func, theta_grid, r_grid, field_type='Radial'):
    normals, dx_dtheta, dx_dr, x, y = compute_surface_normals_and_basis(param_func, theta_grid, r_grid)
    x_vals, y_vals, z_vals = param_func(theta_grid, r_grid)

    if field_type == 'Radial':
        # Vector field is defined globally in ℝ³ and then projected to tangent space
        raw_field = radial_vector_field(x_vals, y_vals, z_vals)
        projected = project_to_surface(raw_field, normals)
    else:
        # Vector field defined in local (θ, r) coordinates and pushed forward
        if field_type == 'Vortex':
            v_theta, v_r = field_vortex(theta_grid, r_grid)
        elif field_type == 'Spiral':
            v_theta, v_r = field_spiral(theta_grid, r_grid)
        elif field_type == 'Petal':
            v_theta, v_r = field_petal(theta_grid, r_grid, n=6)
        elif field_type == 'Shear':
            v_theta, v_r = field_shear(theta_grid, r_grid)
        elif field_type == 'Gaussian Vortex':
            v_theta, v_r = field_gaussian_vortex(theta_grid, r_grid)
        elif field_type == 'Twist':
            v_theta, v_r = field_twist(theta_grid, r_grid)
        else:
            raise ValueError(f"Unknown vector field type: {field_type}")

        projected = pushforward_parametric_field(v_theta, v_r, dx_dtheta, dx_dr)

    # Project the 3D vector field onto the local basis to get scalar components
    proj_theta = np.sum(projected * dx_dtheta, axis=-1)
    proj_r = np.sum(projected * dx_dr, axis=-1)

    # Compute numerical gradients to approximate divergence
    d_proj_theta = np.gradient(proj_theta, axis=1)  # θ-direction
    d_proj_r = np.gradient(proj_r, axis=0)          # r-direction
    divergence = d_proj_theta + d_proj_r            # divergence scalar field

    return x, y, divergence


In [22]:
# ------------------------------------------------------------------------------
# plot_divergence_projection(...)
#
# Displays a 2D color contour plot of a scalar divergence field computed
# over a surface. Used to visualize topological and geometric features
# of vector fields defined on 2D manifolds embedded in ℝ³.
#
# Features:
# - Optionally overlays boundary curve (knot)
# - Supports color normalization and colormap selection
# - Can toggle axes, colorbar, and contour lines
#
# Mathematical & Visual Context:
# - The divergence scalar field can indicate sources/sinks or conserved flow regions.
# - Contour lines act as isolines for divergence level sets, analogous to elevation maps.
#
# References:
# - Marsden and Tromba, "Vector Calculus", Ch. 6 (Divergence and visual interpretation)
# - Cantarella, DeTurck, Gluck, "Vector Calculus and the Geometry of Curl and Divergence"
# - Lee, J. M., "Smooth Manifolds", for the scalar field view of div(V)
# ------------------------------------------------------------------------------

def plot_divergence_projection(
    x, y, divergence,
    title="Surface Divergence Field",
    colormap='plasma',
    contour_levels=60,
    show_contours=True,
    hide_axes=True,
    outline_coords=None,
    outline_color='black',
    show_colorbar=True,
    figsize=(8, 8),
    norm=None
):
    fig, ax = plt.subplots(figsize=figsize)
    ax.set_aspect('equal')

    # Filled color contours for scalar field
    c = ax.contourf(x, y, divergence, levels=contour_levels, cmap=colormap, norm=norm)

    # Optional overlay of knot boundary
    if outline_coords is not None:
        x_b, y_b = outline_coords
        ax.plot(x_b, y_b, color=outline_color, linewidth=2.5, label="Boundary")

    # Optional isolines
    if show_contours:
        ax.contour(x, y, divergence, levels=contour_levels, colors='k', linewidths=0.3)

    if hide_axes:
        ax.axis('off')
    else:
        ax.set_xlabel("x")
        ax.set_ylabel("y")

    if show_colorbar:
        plt.colorbar(c, ax=ax, label="Divergence")
    plt.tight_layout()
    plt.show()


In [32]:
# ------------------------------------------------------------------------------
# INTERACTIVE INTERFACE FOR TOPOLOGICAL FIELD VISUALIZATION
#
# This section defines a Jupyter-based graphical interface using ipywidgets
# that allows users to interactively:
#   - Select a knot surface (Seifert surface) from the built-in library
#   - Choose a vector field defined on the surface
#   - Adjust visualization settings such as contour levels, colormap, 
#     divergence scaling (vmin/vmax), and optional rotation
#
# On user input, the interface computes the divergence of the selected
# vector field on the selected surface and plots the result.
#
# The goal is to create an exploratory tool that links topology, geometry,
# and vector field behavior in an intuitive and visually engaging way.
#
# References:
# - Munzner, T. "Visualization Analysis and Design" (interactive data exploration)
# - Abraham, Marsden, Ratiu: "Manifolds, Tensor Analysis, and Applications"
# - Lee, J. M., "Introduction to Smooth Manifolds", Ch. 8 (vector fields)
# ------------------------------------------------------------------------------

# --- User Controls ---

# Dropdown for selecting the surface (typically a Seifert surface for a knot)
surface_dropdown = widgets.Dropdown(
    options=['3_1', '4_1', '5_1', '5_2', '6_1', '7_1', '8_1', '10_1', '12_1', 
             'T(2,3)', 'T(3,5)', 'T(4,7)', 'T(5,8)', 'T(6,11)', 'T(7,10)'],
    value='3_1',
    description='Surface:'
)

# Dropdown to select vector field type
field_dropdown = widgets.Dropdown(
    options=['Radial', 'Vortex', 'Spiral', 'Petal', 'Shear', 'Gaussian Vortex', 'Twist'],
    value='Radial',
    description='Vector Field:'
)

# Dropdown to select the color map used in contour visualization
colormap_dropdown = widgets.Dropdown(
    options=plt.colormaps(),
    value='plasma',
    description='Colormap:'
)

# Slider to choose the number of contour levels
contour_slider = widgets.IntSlider(
    value=60,
    min=10,
    max=200,
    step=10,
    description='Contours:'
)

# Controls for minimum/maximum values of divergence color scale
vmin_slider = widgets.FloatSlider(
    value=-1.0,
    min=-5.0,
    max=0.0,
    step=0.1,
    description='vmin:'
)

vmax_slider = widgets.FloatSlider(
    value=1.0,
    min=0.0,
    max=5.0,
    step=0.1,
    description='vmax:'
)

# Optionally rotate the visualization in 2D (for aesthetic control)
rotation_slider = widgets.IntSlider(
    value=0,
    min=-180,
    max=180,
    step=5,
    description='Rotate (°):'
)

# Toggles to control display settings
outline_toggle = widgets.Checkbox(value=True, description='Show Outline')
axis_toggle = widgets.Checkbox(value=True, description='Hide Axes')
contour_toggle = widgets.Checkbox(value=True, description='Show Contour Lines')
colorbar_toggle = widgets.Checkbox(value=True, description='Show Colorbar')

# Button to trigger rendering
plot_button = widgets.Button(description='Generate Plot', button_style='success')
output_area = widgets.Output()

# --- Surface Sampling Grid (in polar coordinates θ and r) ---

# Create a meshgrid in parameter space (θ, r)
res_theta, res_r = 300, 100
theta_vals = np.linspace(0, 2 * np.pi, res_theta)
r_vals = np.linspace(0, 1, res_r)
theta_grid, r_grid = np.meshgrid(theta_vals, r_vals)

# --- Callback: Re-render plot on user interaction ---

def on_plot_button_click(b):
    with output_area:
        output_area.clear_output()

        # Gather user selections
        surface_id = surface_dropdown.value
        cmap = colormap_dropdown.value
        n_levels = contour_slider.value
        show_outline = outline_toggle.value
        show_contours = contour_toggle.value
        show_colorbar = colorbar_toggle.value
        hide_axes = axis_toggle.value
        vmin = vmin_slider.value
        vmax = vmax_slider.value
        rot_angle = np.radians(rotation_slider.value)

        # Generate the selected surface
        param_func = lambda th, r: get_surface_parametrization(surface_id, th, r)
        outline = param_func(theta_vals, np.ones_like(theta_vals)) if show_outline else None
        x_b, y_b = outline[:2] if outline is not None else (None, None)

        # Compute divergence of selected vector field on the surface
        field_type = field_dropdown.value
        x, y, div = compute_projected_divergence(param_func, theta_grid, r_grid, field_type=field_type)

        # Optional 2D rotation for visualization
        if rot_angle != 0:
            x_rot = x * np.cos(rot_angle) - y * np.sin(rot_angle)
            y_rot = x * np.sin(rot_angle) + y * np.cos(rot_angle)
            x, y = x_rot, y_rot
            if show_outline and x_b is not None and y_b is not None:
                x_b_rot = x_b * np.cos(rot_angle) - y_b * np.sin(rot_angle)
                y_b_rot = x_b * np.sin(rot_angle) + y_b * np.cos(rot_angle)
                x_b, y_b = x_b_rot, y_b_rot
        outline_coords = (x_b, y_b) if show_outline else None

        # Render the plot
        plot_divergence_projection(
            x, y, div,
            colormap=cmap,
            contour_levels=n_levels,
            show_contours=show_contours,
            hide_axes=hide_axes,
            outline_coords=outline_coords,
            outline_color='black',
            show_colorbar=show_colorbar,
            norm=Normalize(vmin=vmin, vmax=vmax)
        )

# Connect the callback to the button
plot_button.on_click(on_plot_button_click)

# --- Layout and Display ---

# Arrange widgets into rows and columns
ui = widgets.VBox([
    widgets.HBox([surface_dropdown, colormap_dropdown, field_dropdown]),
    widgets.HBox([contour_slider, outline_toggle, axis_toggle, contour_toggle, colorbar_toggle]),
    plot_button,
    output_area
])

# Add sliders for color scaling and rotation
ui.children += (widgets.HBox([vmin_slider, vmax_slider, rotation_slider]),)

# Display the interactive UI in the notebook
display(ui)

# ------------------------------------------------------------------------------
# SAVE TO PNG FUNCTIONALITY
#
# This section adds export capability to the interactive interface.
# The user can:
#   - Enter a desired filename (e.g., "my_knot_plot.png")
#   - Click a "Save as PNG" button
#   - Save the current vector field divergence plot to a high-res image file
#
# Purpose:
# - Supports documentation of results for research posters, presentations, or publications
# - Enables students and users to create artistic or pedagogical exports of
#   topological field visualizations
#
# Notes:
# - This reuses the latest user-selected parameters (surface, field, color options)
# - It regenerates the figure from scratch using the same computation pipeline
#
# References:
# - Munzner, T. "Visualization Analysis and Design"
# - Matplotlib documentation: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.savefig.html
# - Standard usage in computational geometry and math visualization tools
# ------------------------------------------------------------------------------

# --- Button to trigger PNG export ---
save_button = widgets.Button(
    description='Save as PNG',
    button_style='info'  # Blue style for neutral action
)

# --- Text input to let the user name the output file ---
filename_text = widgets.Text(
    value='knot_plot.png',
    description='Filename:'
)

# --- Save callback function: regenerates the plot and writes to disk ---

def on_save_button_click(b):
    with output_area:
        output_area.clear_output(wait=True)

        # Pull latest UI settings
        surface_id = surface_dropdown.value
        cmap = colormap_dropdown.value
        n_levels = contour_slider.value
        show_outline = outline_toggle.value
        show_contours = contour_toggle.value
        show_colorbar = colorbar_toggle.value
        hide_axes = axis_toggle.value

        # Re-generate surface and divergence field
        param_func = lambda th, r: get_surface_parametrization(surface_id, th, r)
        outline = param_func(theta_vals, np.ones_like(theta_vals)) if show_outline else None
        x_b, y_b = outline[:2] if outline is not None else (None, None)

        x, y, div = compute_projected_divergence(param_func, theta_grid, r_grid)
        outline_coords = (x_b, y_b) if show_outline else None

        # Set up figure
        fig, ax = plt.subplots(figsize=(8, 8))
        ax.set_aspect('equal')
        c = ax.contourf(x, y, div, levels=n_levels, cmap=cmap)

        # Optional overlay of the boundary outline
        if outline_coords is not None:
            ax.plot(x_b, y_b, color='black', linewidth=2.5)

        # Optional contour lines
        if show_contours:
            ax.contour(x, y, div, levels=n_levels, colors='k', linewidths=0.3)

        # Optional axes display
        if hide_axes:
            ax.axis('off')
        else:
            ax.set_xlabel("x")
            ax.set_ylabel("y")

        # Optional colorbar
        if show_colorbar:
            plt.colorbar(c, ax=ax, label="Divergence")

        # Save the figure to the specified file
        plt.tight_layout()
        plt.savefig(filename_text.value, dpi=300, bbox_inches='tight')
        plt.close(fig)
        print(f"Plot saved as {filename_text.value}")

# Link the save function to the button
save_button.on_click(on_save_button_click)

# Add the filename input and save button to the user interface
ui.children += (widgets.HBox([filename_text, save_button]),)
